In [1]:
from rdflib import Graph

In [2]:
g = Graph()
g.parse("../rdf/lw-eii.ttl", format="ttl")

<Graph identifier=N9e758050568546e4b1e5e1e52c41f966 (<class 'rdflib.graph.Graph'>)>

In [3]:
len(g) # prints 2

import pprint
for stmt in g:
    pprint.pprint(stmt)


(rdflib.term.URIRef('http://id.csiro.au/dataset/lw-eii/SISSvoc-explorer'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
 rdflib.term.Literal('SISSvoc explorer'))
(rdflib.term.URIRef('http://id.csiro.au/project/BA'),
 rdflib.term.URIRef('http://www.w3.org/ns/prov#used'),
 rdflib.term.URIRef('http://id.csiro.au/dataset/lw-eii/RVA'))
(rdflib.term.URIRef('http://id.csiro.au/person/peter.fitch'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'))
(rdflib.term.URIRef('http://id.csiro.au/org/DOEE'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/ns/org#FormalOrganization'))
(rdflib.term.URIRef('http://id.csiro.au/person/simon.cox'),
 rdflib.term.URIRef('http://www.w3.org/ns/org#memberOf'),
 rdflib.term.URIRef('http://id.csiro.au/orgunit/EII'))
(rdflib.term.URIRef('http://id.csiro.au/person/peter.fitch'),
 rdflib.term.URIRef('http:/

In [4]:
qres = g.query(
    """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT DISTINCT ?x ?p ?y ?xtype ?ytype
       WHERE {
          ?a ?p ?b .
          ?a rdfs:label ?x .
          ?a a ?xtype .
          ?b rdfs:label ?y .
          ?b a ?ytype .
       }""")
print(qres)


In [5]:
thingIdx = {}     #dict of things
thingTypeIdx = {} #dict of array of things
thingTypeReverseIdx = {} #dict of array of things

relationList = []  #dict of relationships between things

for (x,p,y,xtype,ytype) in qres:
    print(x)
    print(g.qname(xtype))
    subject = str(x)
    subject_type = str(g.qname(xtype))
    
    print(g.qname(p))
    prop = str(g.qname(p))
    
    print(y)
    print(g.qname(ytype))
    obj = str(y)
    obj_type = str(g.qname(ytype))
    print("\n")
    
    #add things to thingIdx to ensure unique things
    if(subject not in thingIdx):
        thingIdx[subject] = 1
    else:
        #might as well increment count of occurences
        thingIdx[subject] = thingIdx[subject] + 1
    
    if(obj not in thingIdx):
        thingIdx[obj] = 1
    else:
        #might as well increment count of occurences
        thingIdx[obj] = thingIdx[obj] + 1
    
    #add things to thingTypeIdx
    if(subject_type not in thingTypeIdx):
        thingTypeIdx[subject_type] = {}
    thingTypeIdx[subject_type][subject] = 1
    
    if(subject not in thingTypeReverseIdx):
        thingTypeReverseIdx[subject] = {}
    thingTypeReverseIdx[subject][subject_type] = 1
    
    if(obj_type not in thingTypeIdx):
        thingTypeIdx[obj_type] = {}
    thingTypeIdx[obj_type][obj] = 1
    
    if(obj not in thingTypeReverseIdx):
        thingTypeReverseIdx[obj] = {}
    thingTypeReverseIdx[obj][obj_type] = 1
    
    #add relations
    rel = {
        "source" : subject,
        "target" : obj,
        "value"  : '1',
        "type"   :  prop
    }
    relationList.append(rel)
    

Ben Leighton
prov:Person
org:memberOf
Environmental Information Infrastructure team
foaf:Group


Ben Leighton
prov:Person
org:memberOf
Environmental Information Infrastructure team
prov:Organization


Ben Leighton
prov:Person
org:memberOf
Environmental Information Infrastructure team
org:OrganizationalUnit


W3C Data Catalog - revision
eii:Technology
rdf:type
Technology or component
owl:Class


W3C Data Catalog - revision
eii:Technology
prov:wasAttributedTo
World Wide Web Consortium
prov:Organization


W3C Data Catalog - revision
eii:Technology
prov:wasAttributedTo
World Wide Web Consortium
org:FormalOrganization


W3C Data Catalog - revision
eii:Technology
prov:wasDerivedFrom
W3C Data Catalog
eii:Technology


W3C Data Catalog - revision
eii:Technology
prov:wasAttributedTo
W3C Data eXchange Working Group
org:Organization


W3C Data Catalog - revision
eii:Technology
prov:wasAttributedTo
W3C Data eXchange Working Group
prov:Organization


AEKOS ontology
proj:Project
prov:used
Semantic We

Linked data registry 2
eii:Technology
dcterms:requires
Linked Data Registry
eii:Technology


Knowledge Network
eii:Technology
prov:wasAttributedTo
Environmental Information Infrastructure team
foaf:Group


Knowledge Network
eii:Technology
rdf:type
Technology or component
owl:Class


Knowledge Network
eii:Technology
prov:wasAttributedTo
Environmental Information Infrastructure team
prov:Organization


Knowledge Network
eii:Technology
prov:wasAttributedTo
Environmental Information Infrastructure team
org:OrganizationalUnit


SSN extensions
eii:Technology
rdf:type
Technology or component
owl:Class


SSN extensions
eii:Technology
prov:wasDerivedFrom
W3C Sensors Observations Sampling and Actuation vocabulary
eii:Technology


SSN extensions
eii:Technology
prov:wasDerivedFrom
W3C Semantic Sensor Network Ontology
eii:Technology


SSN extensions
eii:Technology
prov:wasAttributedTo
Simon Cox
prov:Person


Project DAMbuster
proj:Project
proj:hasParticipant
Ben Leighton
prov:Person


Project DAMbu

W3C Data eXchange Working Group
org:Organization
org:unitOf
World Wide Web Consortium
org:FormalOrganization


Linked Data Registry
eii:Technology
prov:wasAttributedTo
Epimorphics 
prov:Organization


Linked Data Registry
eii:Technology
rdf:type
Technology or component
owl:Class


Linked Data Registry
eii:Technology
prov:wasAttributedTo
Epimorphics 
org:FormalOrganization


Commonwealth Scientific and Industrial Research Organisation
prov:Organization
org:hasUnit
Land and Water business unit
prov:Organization


Commonwealth Scientific and Industrial Research Organisation
prov:Organization
org:hasUnit
Land and Water business unit
org:OrganizationalUnit


Commonwealth Scientific and Industrial Research Organisation
prov:Organization
org:hasUnit
Energy business unit
org:OrganizationalUnit


Commonwealth Scientific and Industrial Research Organisation
prov:Organization
org:hasUnit
Energy business unit
prov:Organization


Ashlin Lee
prov:Person
org:memberOf
Environmental Information Infrast

Commonwealth Scientific and Industrial Research Organisation
prov:Organization


CSIRO Data Governance
proj:Project
proj:hasSponsor
Commonwealth Scientific and Industrial Research Organisation
org:FormalOrganization


Linked data registry 1
eii:Technology
rdf:type
Technology or component
owl:Class


Linked data registry 1
eii:Technology
dcterms:requires
Linked Data Registry
eii:Technology


Simon Cox
prov:Person
org:headOf
Environmental Information Infrastructure team
foaf:Group


Simon Cox
prov:Person
org:memberOf
Environmental Information Infrastructure team
foaf:Group


Simon Cox
prov:Person
org:headOf
Environmental Information Infrastructure team
prov:Organization


Simon Cox
prov:Person
org:memberOf
Environmental Information Infrastructure team
prov:Organization


Simon Cox
prov:Person
org:headOf
Environmental Information Infrastructure team
org:OrganizationalUnit


Simon Cox
prov:Person
org:memberOf
Environmental Information Infrastructure team
org:OrganizationalUnit


CSIRO Netw

In [6]:
pprint.pprint(thingIdx)

{'5-star data sharing assessment': 5,
 'AEKOS ontology': 10,
 'ASRIS': 5,
 'Ashlin Lee': 5,
 'Atlas of Living Australia': 4,
 'AuScope': 4,
 'Australian Bureau of Statistics': 2,
 'Australian National Data Service': 10,
 'Ben Leighton': 6,
 'Biodiversity Information Standards (TDWG)': 2,
 'Bioregional Assessments': 6,
 'CKAN': 5,
 'CODATA Commission on Standards': 3,
 'CRISP': 1,
 'CSIRO Data Access Portal': 1,
 'CSIRO Data Governance': 4,
 'CSIRO Data Ratings': 6,
 'CSIRO Network': 1,
 'Commonwealth Scientific and Industrial Research Organisation': 16,
 'DCAT plugin for CKAN': 2,
 'Darwin Core': 4,
 'David Lemon': 7,
 'Department of the Environment and Energy': 2,
 'EPBC': 2,
 'Earth Science Information Partners': 22,
 'Earth and Environment Science Information Partners': 36,
 'Energy Use Data Model - Workstream 5': 6,
 'Energy business unit': 4,
 'Environment Ontology': 2,
 'Environmental Informatics group': 12,
 'Environmental Information Infrastructure team': 39,
 'Epimorphics ': 2

In [7]:
pprint.pprint(thingTypeIdx)

{'eii:Technology': {'5-star data sharing assessment': 1,
                    'CKAN': 1,
                    'CSIRO Data Access Portal': 1,
                    'CSIRO Data Ratings': 1,
                    'CSIRO Network': 1,
                    'DCAT plugin for CKAN': 1,
                    'Darwin Core': 1,
                    'Environment Ontology': 1,
                    'Excel2LDR': 1,
                    'Findable, Accessible, Interoperable & Reusable': 1,
                    'Knowledge Network': 1,
                    'Linked Data Registry': 1,
                    'Linked data registry 1': 1,
                    'Linked data registry 2': 1,
                    'OBO Foundry biomedical ontologies': 1,
                    'OGC Observations and Measurements': 1,
                    'PROMS': 1,
                    'Profiling standards - description': 1,
                    'RDF4J - formerly known as Sesame': 1,
                    'Research Data Australia': 1,
                    'Rese

In [8]:
pprint.pprint(thingTypeReverseIdx)

{'5-star data sharing assessment': {'eii:Technology': 1},
 'AEKOS ontology': {'proj:Project': 1},
 'ASRIS': {'proj:Project': 1},
 'Ashlin Lee': {'prov:Person': 1},
 'Atlas of Living Australia': {'org:FormalOrganization': 1,
                               'prov:Organization': 1},
 'AuScope': {'org:FormalOrganization': 1, 'prov:Organization': 1},
 'Australian Bureau of Statistics': {'org:FormalOrganization': 1,
                                     'prov:Organization': 1},
 'Australian National Data Service': {'org:FormalOrganization': 1,
                                      'prov:Organization': 1},
 'Ben Leighton': {'prov:Person': 1},
 'Biodiversity Information Standards (TDWG)': {'org:Organization': 1,
                                               'prov:Organization': 1},
 'Bioregional Assessments': {'proj:Project': 1},
 'CKAN': {'eii:Technology': 1},
 'CODATA Commission on Standards': {'proj:Project': 1},
 'CRISP': {'proj:Project': 1},
 'CSIRO Data Access Portal': {'eii:Technology': 

In [9]:
pprint.pprint(relationList)

[{'source': 'Ben Leighton',
  'target': 'Environmental Information Infrastructure team',
  'type': 'org:memberOf',
  'value': '1'},
 {'source': 'Ben Leighton',
  'target': 'Environmental Information Infrastructure team',
  'type': 'org:memberOf',
  'value': '1'},
 {'source': 'Ben Leighton',
  'target': 'Environmental Information Infrastructure team',
  'type': 'org:memberOf',
  'value': '1'},
 {'source': 'W3C Data Catalog - revision',
  'target': 'Technology or component',
  'type': 'rdf:type',
  'value': '1'},
 {'source': 'W3C Data Catalog - revision',
  'target': 'World Wide Web Consortium',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'W3C Data Catalog - revision',
  'target': 'World Wide Web Consortium',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'W3C Data Catalog - revision',
  'target': 'W3C Data Catalog',
  'type': 'prov:wasDerivedFrom',
  'value': '1'},
 {'source': 'W3C Data Catalog - revision',
  'target': 'W3C Data eXchange Working Group'

  'value': '1'},
 {'source': 'Darwin Core',
  'target': 'Technology or component',
  'type': 'rdf:type',
  'value': '1'},
 {'source': 'Darwin Core',
  'target': 'Biodiversity Information Standards (TDWG)',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'Darwin Core',
  'target': 'Biodiversity Information Standards (TDWG)',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'CKAN',
  'target': 'Technology or component',
  'type': 'rdf:type',
  'value': '1'},
 {'source': 'CKAN',
  'target': 'Open Knowledge International',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'CKAN',
  'target': 'Open Knowledge International',
  'type': 'prov:wasAttributedTo',
  'value': '1'},
 {'source': 'David Lemon',
  'target': 'Environmental Informatics group',
  'type': 'org:headOf',
  'value': '1'},
 {'source': 'David Lemon',
  'target': 'Environmental Informatics group',
  'type': 'org:headOf',
  'value': '1'},
 {'source': 'CSIRO Data Ratings',
  'target': 'Env

In [27]:
#create index out of types
#get types
typeList = thingTypeIdx.keys()
typeIdx = {}
nodes = []
counter = 1
for t in typeList:
    typeIdx[t] = counter
    counter = counter + 1
    
for obj in thingTypeReverseIdx.keys():
    typeIds = []
    arrTypeLabels = []
    for x in thingTypeReverseIdx[obj].keys():
        typeIds.append(typeIdx[x])
        arrTypeLabels.append(x)
    #obj = { "id" : id, "group": typeIds }
    #obj = { "id" : id, "group": arrTypeLabels }
    group = 1
    if('prov:Person' in arrTypeLabels):
        group = 10
    if('prov:Organization' in arrTypeLabels):
        group = 5
    if('eii:Technology' in arrTypeLabels):
        group = 7
        
    d = { "id" : obj, "group": group }
    
    nodes.append(d)

pprint.pprint(nodes)
pprint.pprint(typeList)
pprint.pprint(typeIdx)
               

[{'group': 10, 'id': 'Ben Leighton'},
 {'group': 5, 'id': 'Environmental Information Infrastructure team'},
 {'group': 7, 'id': 'W3C Data Catalog - revision'},
 {'group': 1, 'id': 'Technology or component'},
 {'group': 5, 'id': 'World Wide Web Consortium'},
 {'group': 7, 'id': 'W3C Data Catalog'},
 {'group': 5, 'id': 'W3C Data eXchange Working Group'},
 {'group': 1, 'id': 'AEKOS ontology'},
 {'group': 7, 'id': 'Semantic Web for Earth and Environmental Terminology '},
 {'group': 7, 'id': 'Simple Knowledge Organization System'},
 {'group': 7,
  'id': 'W3C Sensors Observations Sampling and Actuation vocabulary'},
 {'group': 7, 'id': 'W3C Semantic Sensor Network Ontology'},
 {'group': 7, 'id': 'SSN extensions'},
 {'group': 7, 'id': 'Darwin Core'},
 {'group': 5, 'id': 'Terrestrial Ecosystems Research Network'},
 {'group': 7, 'id': 'Environment Ontology'},
 {'group': 10, 'id': 'Simon Cox'},
 {'group': 7, 'id': 'PROMS'},
 {'group': 1, 'id': 'Bioregional Assessments'},
 {'group': 7, 'id': 'W3C

In [28]:
d3_obj = { "nodes" : nodes, "links" : relationList}
pprint.pprint(d3_obj)

{'links': [{'source': 'Ben Leighton',
            'target': 'Environmental Information Infrastructure team',
            'type': 'org:memberOf',
            'value': '1'},
           {'source': 'Ben Leighton',
            'target': 'Environmental Information Infrastructure team',
            'type': 'org:memberOf',
            'value': '1'},
           {'source': 'Ben Leighton',
            'target': 'Environmental Information Infrastructure team',
            'type': 'org:memberOf',
            'value': '1'},
           {'source': 'W3C Data Catalog - revision',
            'target': 'Technology or component',
            'type': 'rdf:type',
            'value': '1'},
           {'source': 'W3C Data Catalog - revision',
            'target': 'World Wide Web Consortium',
            'type': 'prov:wasAttributedTo',
            'value': '1'},
           {'source': 'W3C Data Catalog - revision',
            'target': 'World Wide Web Consortium',
            'type': 'prov:wasAttributedTo',


           {'source': 'ASRIS',
            'target': 'Linked Data Registry',
            'type': 'prov:used',
            'value': '1'},
           {'source': 'ASRIS',
            'target': 'Linda Gregory',
            'type': 'proj:hasParticipant',
            'value': '1'},
           {'source': 'ASRIS',
            'target': 'Simon Cox',
            'type': 'proj:hasParticipant',
            'value': '1'},
           {'source': 'Water Resource Management program',
            'target': 'Environmental Informatics group',
            'type': 'org:hasUnit',
            'value': '1'},
           {'source': 'Water Resource Management program',
            'target': 'Environmental Informatics group',
            'type': 'org:hasUnit',
            'value': '1'},
           {'source': 'W3C Data eXchange Working Group',
            'target': 'World Wide Web Consortium',
            'type': 'org:unitOf',
            'value': '1'},
           {'source': 'W3C Data eXchange Working Group',
     

In [29]:
import json
with open('data.json', 'w') as outfile:
    json.dump(d3_obj, outfile, sort_keys = True, indent = 4)